<a href="https://colab.research.google.com/github/maayanb578/YOLO_Car_Detection/blob/main/YOLO_Car_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -qr https://raw.githubusercontent.com/ultralytics/yolov5/master/requirements.txt  # install dependencies


     |████████████████████████████████| 182 kB 8.1 MB/s 
     |████████████████████████████████| 62 kB 1.2 MB/s 
     |████████████████████████████████| 1.6 MB 56.4 MB/s 


# Import Library 

In [13]:
import torch
import cv2
import random
import os
import matplotlib.pyplot as plt
import numpy as np
import json 
from google.colab.patches import cv2_imshow
import glob

# Setting Variables

In [3]:
height = 256
width = 256
img_size = (height, width)
writer = None
H, W = None, None

# Importing a pre-trained Yolo model and testing it on a single image

In [4]:

# Model
model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)
model.conf = 0.73

# Images
imgs = ['https://images.pexels.com/photos/20411/pexels-photo.jpg']  # batch of images

# Inference
results = model(imgs)
# Results
print("test")
#results.print()
print(results)
print("tets")
results.save()  # or .show()

results.xyxy[0]  # img1 predictions (tensor)
results.pandas().xyxy[0]  # img1 predictions (pandas)

/usr/local/lib/python3.8/dist-packages/torch/hub.py:266: UserWarning: You are about to download and run code from an untrusted repository. In a future release, this won't be allowed. To add the repository to your trusted list, change the command to {calling_fn}(..., trust_repo=False) and a command prompt will appear asking for an explicit confirmation of trust, or load(..., trust_repo=True), which will assume that the prompt is to be answered with 'yes'. You can also use load(..., trust_repo='check') which will only prompt for confirmation if the repo is not already trusted. This will eventually be the default behaviour
  warnings.warn(
Downloading: "https://github.com/ultralytics/yolov5/zipball/master" to /root/.cache/torch/hub/master.zip
YOLOv5 🚀 2022-12-4 Python-3.8.15 torch-1.12.1+cu113 CPU



  0%|          | 0.00/14.1M [00:00<?, ?B/s]


Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients
Adding AutoShape... 
Saved 1 image to runs/detect/exp


test
image 1/1: 1212x1900 1 car
Speed: 327.8ms pre-process, 362.9ms inference, 17.2ms NMS per image at shape (1, 3, 416, 640)
tets


,xmin,ymin,xmax,ymax,confidence,class,name
0,461.353943,484.085175,1460.828003,871.343201,0.907498,2,car


# Predicting objects on a video file

In [6]:

cap = cv2.VideoCapture("/content/drive/MyDrive/YOLO_Car_Detection/traffic.mp4")

# Check if camera opened successfully
if (cap.isOpened()== False):
    print("Error opening video file")
 

out_dir = "./resimages3"
os.makedirs(out_dir, exist_ok=True)
# Read until video is completed
json_results = {}
frame_num = 1
while(cap.isOpened()):
   
  # Capture frame-by-frame
  ret, frame = cap.read()
    # get frame dimensions
  if W is None or H is None: ##
      (H, W) = frame.shape[:2] ##

  
  frame_results = {}
  if ret == True:
      results1 = model(frame)
      counter = 1
      for res in results1.xyxy[0]:
        res0 = res.numpy()
        if res0[5] != 2 or res0[4] < 0.73:
          continue
        frame_results[f"object{counter}"] = [str(x) for x in list(res0[:4])]
        counter += 1
        
      img = np.squeeze(results1.render(labels=[2]))
      cv2.imwrite(os.path.join(out_dir, f"frame{frame_num}.jpg"), img)
      json_results[f"frame{frame_num}"] = frame_results
        
  frame_num += 1 

  # Break the loop
  if not ret:
    break
 
# When everything done, release
# the video capture object
cap.release()
 
# Closes all the frames
cv2.destroyAllWindows()

# Sort frames in order

In [9]:
t = list(glob.glob('./resimages3/*.jpg'))

In [10]:
def sort_int(x):
  r = x[18:-4]
  return int(r)

ts = sorted(t, key=sort_int)

# Creating a video file after identifying and marking the objects

In [11]:

out = cv2.VideoWriter('output_video.avi',cv2.VideoWriter_fourcc(*'XVID'), 30, (W, H), True)

for filename in ts:
    img = cv2.imread(filename)
    out.write(img)

out.release()

# Export object data to a JSON file


In [12]:
with open("results.json", "w") as outfile:
  json.dump(json_results, outfile)